In [66]:
# import modules
import os
import shutil
from datetime import date

In [1]:
'''Copying and renaming files'''

def bidsify(origpath, destpath, n_sessions=2, scan_types=None, detect_size=True, log_changes=True, log_name='CHANGES', verbose=False):
    
    
    # NOTE: mapping old names to new names by passing a dict is not currently supported (scan_names kwarg).
    # detect_size must be True (default) to work. Passing custom file size ranges not currently supported (file_sizes kwarg)
    
    # convert to absolute paths if relative path is passed
    origpath_abs = os.path.abspath(origpath)
    destpath_abs = os.path.abspath(destpath)
    
    # dict to hold unsuccessfully handled files with errors
    problem_files = {}
    
    # default output folder names
    if scan_types is None:
        scan_types = ['anat','func','log']
    
    # for name mapping based on file size
    if detect_size:    #and scan_names is None:
        method = 'size'
        
    else:
        raise ValueError('detect_size must be True. Mapping by other file attributes not currently supported.')
    
    # for name mapping based on passed names
#     elif not detect_size and scan_names is None:
#         # use default dictionary of scan types
#         scan_names = {
#             'mprage' : 'T1w',
#             'bold1' : 'task-rest_run-01_bold',
#             'bold2' : 'task-mcr_run-02_bold',
#             'bold3' : 'task-swm_run-03_bold',
#             'bold4' : 'task-dd_run-04_bold',
#             'bold5' : 'task-rest_run-05_bold'
#             }
#         method = 'name'
    
#     elif detect_size and scan_names:
#         raise ValueError('Cannot map by both scan_names and detect_size')
        
#     if scan_names and type(scan_names) is not dict:
#         raise TypeError('scan_names must be a dict')

    for i, (root, dirs, files) in enumerate(os.walk(origpath_abs)):
        # create new directory structure
        if i == 0:
            [os.makedirs(destpath_abs+'/'+direc.split('_')[0]+'/ses-'+str(ses+1)+'/'+scantype, exist_ok=True) for direc in dirs for scantype in scan_types for ses in range(n_sessions)]
        
        # move and rename files
        else:
            file_list = [f for f in files if not f.startswith('.')]
            if file_list:
                if method == 'size':
                    path_map, prob_fs = _rename_size(file_list, root, n_sessions, destpath_abs, log_changes, log_name, verbose)
                    
                    for old_filepath, new_filepath in path_map.items():
                        try:
                            shutil.copy(old_filepath, new_filepath)
                            
                            if verbose:
                                print('moved ' + olf_filepath + ' to ' + new_filepath)
                        
                        # new_filepath is None if error occurs
                        except TypeError:
                            pass
                            
                    if prob_fs:
                        problem_files.update(prob_fs)
                                
#                 elif method == 'name':
#                     for file in file_list:
#                         old_filepath = os.path.join(root, file)
#                         new_filepath, problem_file = _rename_dict(file, root, n_sessions, scan_names, destpath_abs, log_changes, log_name, verbose)

#                         if new_filepath is not None:
#                             # move and rename
#                             shutil.copy(old_filepath, new_filepath)

#                             if verbose:
#                                 print('moved ' + old_filepath + ' to ' + new_filepath)

#                         else:
#                             problem_files.append(problem_file)       
                    
                    
    if problem_files:
        print('The following files were not successfully converted: ' + problem_files)

SyntaxError: invalid syntax (<ipython-input-1-18672f7d9878>, line 58)

In [5]:
# def _rename_dict(file, root, n_sessions, scan_names, destpath_abs, log_changes, log_name, verbose):

#     # viarable to track unsuccessfully renamed files
#     problem_file = None

#     old_path = os.path.join(root, file)
#     base, ext = os.path.splitext(file)
#     splitpath = root.split('/')

#     # get subject ID
#     sub = splitpath[-2].split('_')[0]

#     # get session number
#     ses_number = splitpath[-2].split('_')[1]
#     if int(ses_number) <= n_sessions:
#         session = 'ses-'+ses_number

#     else:
#         print('unrecognized session number \'' + ses_number + '\' for subID ' + sub)
#         problem_file = file

#     # get scan type (or log)
#     if splitpath[-1] == 'ANATOMY':
#         runtype = 'anat'

#     elif splitpath[-1] == 'FUNCTIONAL':
#         runtype = 'func'

#     elif splitpath[-1] == 'LOG':
#         runtype = 'LOG'

#     else:
#         print('unrecognized scan or log folder ' + splitpath[-1] + ' for subID ' + sub)
#         problem_file = file

        
#     # format scan name (or preserve name of log file)
#     if base in scan_names:
#         new_name = scan_names[base]

#     elif ext == '.log':
#         new_name = base

#     else:
#         print('unrecognized scan name ' + base + ' for file ' + file)
#         problem_file = file

#     try:
#         if runtype != 'LOG':
#             new_path = os.path.join(destpath_abs, sub, session, runtype, sub+'_'+new_name+ext)
#         else:
#             new_path = os.path.join(destpath_abs, sub, session, runtype, new_name+ext)
            
#         if log_changes:
#             _writelog(log_name, old_path, destpath_abs, sub, session, new_name, ext, verbose)

#     except NameError:
#         new_path = None

    
#     return new_path, problem_file

In [ ]:
'''Defines naming scheme for moved files based on size'''

def _rename_size(file_list, root, n_sessions, destpath_abs, log_changes, log_name, verbose):
    
    old_fps = [os.path.join(root,file) for file in file_list]
    path_maps = dict.fromkeys(old_fps)
    sizes_dict = {os.stat(file).st_size: file for file in old_fps}
    prob_fs = {}
    splitpath = root.split('/')
    
    # get subject ID
    sub = splitpath[-2].split('_')[0]
    
    # get session number
    ses_number = splitpath[-2].split('_')[1]
    if int(ses_number) <= n_sessions:
        session = 'ses-'+ses_number
        
    else:
        prob_fs[sub]= 'unrecognized session number: ' + ses_number
        return path_maps, prob_fs
        
    
    # deal with anatomical scans
    if splitpath[-1] == 'ANATOMY':
        # expected size for mprage files (bytes)
        right_size = 28836192
        # use file that matches expected size, or if none do, use one closest to expected size
        best_scan = sizes_dict[right_size] if right_size in sizes_dict else sizes_dict[min(sizes_dict.keys(), key=lambda k: abs(k-right_size))]
        
        if right_size not in sizes_dict.keys():
            prob_fs[best_scan.split('/')[-1]] = 'No mprage of expected size. Used closest match: ' + best_scan.split('/')[-1])
        
        new_name = 'T1w.nii'
        path_maps[best_scan] = os.path.join(destpath_abs, sub, session, 'anat', sub+'_T1w.nii')
        
        
    # deal with functional scans
    elif splitpath[-1] == 'FUNCTIONAL':
        rests = []
        mcrs = []
        swms = []
        leftovers = []
        for size, old_file in sizes_dict.items():
            # expected size of resting state scan (+/- 2kb)
            if size in range(110590352, 110594352):
                rests.append(old_file)
                
            # expected size of mcr scan (+/- 2kb)    
            elif size in range(77412752, 77416752):
                mrcs.append(old_file)
            
            # expected size of swm scan (+/- 2kb)
            elif size in range(82942352, 82946352):
                swms.append(old_file) 
            else:
                leftovers.append(old_file)
        
        # map resting state scans
        if not rests:
            prob_fs['Resting state'] = 'No scan files matching expected size for Resting State.'
            
        elif len(rests) == 1:
            prob_fs['Resting state'] = 'Unable to determine which Resting State scan for file: ' + rests[0]
            
        elif len(rests) == 2:
            rest1, rest2 = sorted(rests)[0], sorted(rests)[1]
            path_maps[rest1] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-rest_run-01_bold.nii')
            path_maps[rest2] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-rest_run-05_bold.nii')
        else:
            prob_fs['Resting state'] = 'Unable to identify Resting State 1 vs 2 from choices: ' + rests[0]
                    
        # map mcr scans
        if mcrs:
            path_maps[mcrs[-1]] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-mcr_run-02_bold.nii')
        else:
            prob_fs['MCR'] = 'No scan files matching expected size for MCR'
            
        # map swm scans
        if swms:
            path_maps[swms[-1]] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-swm_run-03_bold.nii')
        else:
            prob_fs['SWM'] = 'No scan files matching expected size for SWM'
 
         
        if len(leftovers) == 1:
            path_maps[leftovers[0]] = os.path.join(destpath_abs, sub, session, 'func', sub+'_task-dd_run-04_bold.nii')
        else:
            prob_fs['DD'] = 'Unable to identify DD scan from choices: ' + str(leftovers)
            
                
                
    elif splitpath[-1] == 'LOG':
        runtype = 'LOG'
        # keep existing name
        for file in old_fps:
            path_maps[file] = file
        
        
    else:
        prob_fs[file_list] = 'unrecognized scan or log folder: ' + splitpath[-1]
        return path_maps, prob_fs
        
        
        
        
    # filter path_maps to remove keys with None values
        
        
        
        
    return filtered_path_maps, prob_fs

In [4]:
'''Writes/updates log of moving file'''

def _writelog(log_name, old_path, destpath_abs, sub, session, new_name, ext, verbose):
    
    filename = os.path.join(destpath_abs, sub, session, 'LOG', log_name + '.log')
    
    if os.path.exists(filename):
        mode = 'a'
    
    else:
        mode = 'w'
        
    with open(filename, mode) as f:
        f.write(
            date.today().strftime('%Y-%m-%d') + '\n' +
            ' - ' + new_name+ext + ' moved from ' + old_path + '\n'
        )
    
    if verbose:
        print('wrote to log file ' + filename)

In [45]:
os.stat('/Users/paxtonfitzpatrick/Desktop/testorig/2018_1_asd/FUNCTIONAL/bold2.txt').st_size

15

In [14]:
# if os.path.isfile('/Users/paxtonfitzpatrick/Desktop/egg6_mapping_tet.egg'):
#     print('success')
# else:
#     raise FileNotFoundError('not there')
#     print(3+4)

In [77]:
for i, (root, dirs, files) in enumerate(os.walk('/Users/paxtonfitzpatrick/Desktop/testorig/')):
    file_list = [f for f in files if not f.startswith('.')]
    if file_list:
        print(file_list)
        print('_______')
        print(root)

    #     print(dirs)
    #     print('_______')
#         print(files)
#         print('_______')

['mprage.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/ANATOMY
['2018_1_asd.log']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/LOG
['bold3.txt', 'bold2.txt', 'bold1.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/FUNCTIONAL
['mprage.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/ANATOMY
['2018_1_asd.log']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/LOG
['bold3.txt', 'bold2.txt', 'bold1.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/FUNCTIONAL
['mprage.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/ANATOMY
['2018_1_asd.log']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/LOG
['bold3.txt', 'bold2.txt', 'bold1.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/FUNCTIONAL
['mprage.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2021_1_asd/ANATOMY
['2018_1_asd.log']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2021_1_

In [67]:
shutil.copy('/Users/paxtonfitzpatrick/Desktop/fuckitpy/', None)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [11]:
testdict = {}

In [14]:
for k,v in testdict.items():
    if k in range(10):
        print('a')
        
    else:
        print('b')

In [16]:
testdict[testvar] = 'test'

NameError: name 'testvar' is not defined

In [28]:
def thingy(l):
    if len(l) == 3:
        print('a')
    elif not l:
        print('b')
    elif len(l) == 9:
        print('c')

In [30]:
allnums = [i for i in range(100)]
evens = [i for i in allnums if i%2==0]
div3s = [i for i in allnums if i%3==0]

In [36]:
[i for i in allnums if i not in evens and i not in div3s]

[1,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 25,
 29,
 31,
 35,
 37,
 41,
 43,
 47,
 49,
 53,
 55,
 59,
 61,
 65,
 67,
 71,
 73,
 77,
 79,
 83,
 85,
 89,
 91,
 95,
 97]

In [38]:
numdict = {i: str(i) for i in range(100)}

In [42]:
[val for key,val in numdict.items() if key not in evens]

['1',
 '3',
 '5',
 '7',
 '9',
 '11',
 '13',
 '15',
 '17',
 '19',
 '21',
 '23',
 '25',
 '27',
 '29',
 '31',
 '33',
 '35',
 '37',
 '39',
 '41',
 '43',
 '45',
 '47',
 '49',
 '51',
 '53',
 '55',
 '57',
 '59',
 '61',
 '63',
 '65',
 '67',
 '69',
 '71',
 '73',
 '75',
 '77',
 '79',
 '81',
 '83',
 '85',
 '87',
 '89',
 '91',
 '93',
 '95',
 '97',
 '99']

In [44]:
l = ['a']

In [45]:
l[0:-1]

[]